In [2]:
import pandas as pd
import numpy as np
import ast
import umap
import plotly.express as px
import plotly.graph_objs as go

/home/adam-kotik/anaconda3/envs/bakalarka/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
embeddings = np.load('../data/svd_vectors.npy')
books = pd.read_csv('books_cleaned.csv')
books['tags'] = books['tags'].apply(ast.literal_eval)

In [4]:
u_2d = umap.UMAP(n_components=2, n_neighbors=250, random_state=42)
umap_2d = u_2d.fit(embeddings)
umap_coords = umap_2d.transform(embeddings)

/home/adam-kotik/anaconda3/envs/bakalarka/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/adam-kotik/anaconda3/envs/bakalarka/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/home/adam-kotik/anaconda3/envs/bakalarka/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [23]:
P = 4
scores_0 = []
for emb in embeddings:
    score = np.exp(-(1 - np.dot(embeddings[250], emb)) * P)
    scores_0.append(score)

scores_0 = np.array(scores_0)
print(len(scores_0))
scores_0 = np.divide(scores_0, np.sum(scores_0))
print(len(scores_0))

9214
9214


In [24]:
scores_0 = (scores_0 - min(scores_0)) / (max(scores_0) - min(scores_0))
print(max(scores_0))
print(min(scores_0))

1.0
0.0


In [25]:
book_75_coord = umap_coords[250]

In [26]:
fig = px.scatter(
    x=umap_coords[:, 0], 
    y=umap_coords[:, 1], 
    color=scores_0,
    color_continuous_scale='Magma', 
    labels={'x': "UMAP 1", 'y': "UMAP 2", 'color': "Scores Intensity"},
    title=f"Score initialization visualization with P = {P}"
)

fig.add_trace(go.Scatter(
    x=[book_75_coord[0]],
    y=[book_75_coord[1]],
    mode='markers',
    marker=dict(
        size=50,
        color='red',
        symbol='star'
    ),
    name="Highlighted Book"
))

fig.update_layout(
    width=1500,
    height=1000,
)

# Show the figure
fig.show()
